# TensorFlow: Static Graphs

- 원본 : [링크](https://pytorch.org/tutorials/beginner/pytorch_with_examples.html)
- Author : [Justin Johnson](https://github.com/jcjohnson/pytorch-examples)
- 번역 & 주석 추가 : [박범진](https://github.com/pbj0812)

PyTorch의 autograd 기능은 TensorFlow를 많이 닮았다: 두 프레임워크에서 우리는 computational graph를 정의하고, gradients를 계산하기 위해 자동 미분을 사용한다. 두 개의 가장 큰 차이점은 TensorFlow의 computational graph는 정적이고 PyTorch는 동적인 computational graph를 사용한다는 것이다.

TensorFlow에서 우리는 computational graph를 먼저 정의 한 다음 다른 입력 데이터가 그래프에 들어올때까지 같은 그래프를 계속 반복해서 실행한다. PyTorch에서는 매 번 새로운 computational graph가 생성된다.

정적인 그래프는 앞에서 그래프를 최적화 할 수 있기에 좋다; 예를 들어 효율성을 위해 여러 그래프들을 융합할수도 있고, 많은 GPU들이나 장치들을 위해 그래프를 분산 배치 할 수도 있다. 동일한 그래프를 반복해서 재사용하는 경우 동일한 그래프가 반복해서 반복 될 때 비용이 많이 드는 초기 최적화를 상환 할 수 있다.

정적인 그래프와 동적인 그래프에서의 다른 측면은 control flow에 있다. 몇 모델에서 우리는 각 데이터 지점에 다른 계산을 수행하고 싶을 수도 있다; 예를 들면 순환 네트워크는 각 데이터 지점의 매 시간마다의 다른 숫자를 위해 풀어져 있다; 이것은 루프로 사용된다. 정적인 그래프를 사용할 때, 루프 구조는 그래프의 일부분이 되어야 한다; 이런 이유로 TensorFlow는 그래프에서의 루프 임베딩을 위해 tf.scan이라는 장치를 제공한다. 다이나믹 그래프를 사용하는 것은 단순하다: 각 예제에 대해 그래프를 즉석에서 작성하기 때문에 정상적인 명령형 흐름 제어를 사용하여 각 입력마다 다른 계산을 수행 할 수 있다.

PyTorchdml autograd와 비교하기 위해, TensorFlow를 이용하여 two-layer net을 제어한다.

In [1]:
# -*- coding: utf-8 -*-
import tensorflow as tf 
import numpy as np

In [2]:
# computational graph 세팅:

In [3]:
N, D_in, H, D_out = 4, 1000, 100, 10

# input 과 target data에 대한 placeholder 정의
# 그래프를 실행할 때 실제 데이터로 실행된다.
x = tf.placeholder(tf.float32, shape=(None, D_in))
y = tf.placeholder(tf.float32, shape=(None, D_out))

# weitght를 위한 Variable 생성 및 무작위수로 초기화
# TensorFlow의 Variable은 그래프의 실행을 통해 값을 유지한다.
w1 = tf.Variable(tf.random_normal((D_in, H)))
w2 = tf.Variable(tf.random_normal((H, D_out)))

In [4]:
# Forward pass : TensorFlow Tensor를 이용하여 y 값 예측.
# 이 코드는 진행되지 않는다.
# 단지 나중의 실행을 위해 computational graph를 세팅 하는 것이다.
h = tf.matmul(x, w1)
h_relu = tf.maximum(h, tf.zeros(1))
y_pred = tf.matmul(h_relu, w2)

In [5]:
# loss 계산 
loss = tf.reduce_sum((y - y_pred) ** 2.0)

In [6]:
# w1과 w2의 loss에 대한 gradient 계산
grad_w1, grad_w2 = tf.gradients(loss, [w1, w2])

In [7]:
# weight 갱신
# weight 갱신을 위해서는 그래프 실행시 새로운 w1과 w2를 평가 할 필요가 있다. 
# PyTorch에서 이 부분을 계산 그래프 바깥쪽에서 진행하였다면, TensorFlow는 내부에서 진행된다.
learning_rate = 1e-6
new_w1 = w1.assign(w1 - learning_rate * grad_w1)
new_w2 = w2.assign(w2 - learning_rate * grad_w2)

In [8]:
# 그래프를 실행시키기 위해서는 session 안에 들어가야 한다.
with tf.Session() as sess:
    # w1과 w2를 초기화 시킨다.
    sess.run(tf.global_variables_initializer())
    
    # numpy 행렬을 만들어 x 와 y를 담는다.
    x_value = np.random.randn(N, D_in)
    y_value = np.random.randn(N, D_out)
    
    since = time.time()
    for i in range(10000):
        # 그래프를 실행하면서 feed_dict를 사용하여 x_value는 x에 y_value는 y에 묶어준다.
        # loss new_w1, new_w2를 구하기 위하여 매번 그래프를 실행한다.
        # 생성된 Tensor 값들은 numpy 행렬로 반환된다.
        loss_value, _, _ = sess.run([loss, new_w1, new_w2],
                                   feed_dict={x: x_value, y: y_value})
        print(i, loss_value)
    now = time.time()

0 2078005.1
1 1083873.6
2 617274.0
3 374431.2
4 238875.83
5 158888.77
6 109438.9
7 77468.734
8 56054.055
9 41274.598
10 30794.855
11 23241.234
12 17702.168
13 13597.402
14 10577.922
15 8383.304
16 6672.6016
17 5330.625
18 4272.2676
19 3433.7742
20 2766.836
21 2234.525
22 1808.3552
23 1466.2295
24 1190.8864
25 968.80994
26 789.3093
27 643.9535
28 526.0444
29 430.24393
30 352.28842
31 288.76266
32 236.93362
33 194.58601
34 159.95569
35 131.59532
36 108.35147
37 89.28208
38 73.62129
39 60.75083
40 50.162117
41 41.445473
42 34.264076
43 28.342726
44 23.458569
45 19.425735
46 16.09406
47 13.340461
48 11.062766
49 9.178021
50 7.6174297
51 6.325131
52 5.2540445
53 4.3658
54 3.6288013
55 3.017486
56 2.510109
57 2.0887046
58 1.7385352
59 1.4472009
60 1.2053039
61 1.0041199
62 0.8366958
63 0.6972561
64 0.58133626
65 0.48468807
66 0.40430665
67 0.33725852
68 0.28138885
69 0.23482767
70 0.19599251
71 0.16362177
72 0.13663273
73 0.11410758
74 0.09531216
75 0.07959487
76 0.06652513
77 0.05559378
78 

563 3.7869526e-08
564 3.7873164e-08
565 3.7377944e-08
566 3.679314e-08
567 3.679314e-08
568 3.679314e-08
569 3.671211e-08
570 3.671211e-08
571 3.6692896e-08
572 3.6978477e-08
573 3.6207226e-08
574 3.6207226e-08
575 3.623269e-08
576 3.623269e-08
577 3.615348e-08
578 3.615348e-08
579 3.615348e-08
580 3.615348e-08
581 3.615348e-08
582 3.599932e-08
583 3.553093e-08
584 3.5410082e-08
585 3.4424616e-08
586 3.4424616e-08
587 3.2486483e-08
588 3.2486483e-08
589 3.246909e-08
590 3.246909e-08
591 3.246909e-08
592 3.281697e-08
593 3.281697e-08
594 3.3126113e-08
595 3.3126113e-08
596 3.3126113e-08
597 3.3126113e-08
598 3.3126113e-08
599 3.255404e-08
600 3.255404e-08
601 3.255404e-08
602 3.2480287e-08
603 3.1191078e-08
604 3.1191078e-08
605 3.1191078e-08
606 3.045848e-08
607 3.045848e-08
608 3.0320237e-08
609 3.0320237e-08
610 3.02483e-08
611 3.02483e-08
612 3.02483e-08
613 3.0281043e-08
614 2.936609e-08
615 2.936609e-08
616 2.936609e-08
617 2.936609e-08
618 2.9295974e-08
619 2.6210285e-08
620 2.60

1129 1.4435001e-08
1130 1.4435001e-08
1131 1.4435001e-08
1132 1.4435001e-08
1133 1.4435001e-08
1134 1.4435001e-08
1135 1.4435001e-08
1136 1.4435001e-08
1137 1.4435001e-08
1138 1.4435001e-08
1139 1.4435001e-08
1140 1.4435001e-08
1141 1.4445005e-08
1142 1.4445005e-08
1143 1.4445005e-08
1144 1.4445005e-08
1145 1.4445005e-08
1146 1.4445005e-08
1147 1.4445005e-08
1148 1.4445005e-08
1149 1.4445005e-08
1150 1.4445005e-08
1151 1.4445005e-08
1152 1.4445005e-08
1153 1.4445005e-08
1154 1.4445005e-08
1155 1.4445005e-08
1156 1.4445005e-08
1157 1.4445005e-08
1158 1.4445005e-08
1159 1.4445005e-08
1160 1.4445005e-08
1161 1.4445005e-08
1162 1.4445005e-08
1163 1.4445005e-08
1164 1.4445005e-08
1165 1.4445005e-08
1166 1.4445005e-08
1167 1.4445005e-08
1168 1.4445005e-08
1169 1.4445005e-08
1170 1.4445005e-08
1171 1.4445005e-08
1172 1.4445005e-08
1173 1.4445005e-08
1174 1.4445005e-08
1175 1.4445005e-08
1176 1.4445005e-08
1177 1.4445005e-08
1178 1.4398166e-08
1179 1.4398166e-08
1180 1.4398166e-08
1181 1.43981

1611 1.3317488e-08
1612 1.355623e-08
1613 1.3317488e-08
1614 1.355623e-08
1615 1.3317488e-08
1616 1.355623e-08
1617 1.3317488e-08
1618 1.355623e-08
1619 1.3308847e-08
1620 1.354759e-08
1621 1.3308847e-08
1622 1.354759e-08
1623 1.3004167e-08
1624 1.3242909e-08
1625 1.3004167e-08
1626 1.3712208e-08
1627 1.2311132e-08
1628 1.3019173e-08
1629 1.2173798e-08
1630 1.2913217e-08
1631 1.2173798e-08
1632 1.3733581e-08
1633 1.2994162e-08
1634 1.3733581e-08
1635 1.2994162e-08
1636 1.3733581e-08
1637 1.2994162e-08
1638 1.3733581e-08
1639 1.2994162e-08
1640 1.3733581e-08
1641 1.2994162e-08
1642 1.3733581e-08
1643 1.29978e-08
1644 1.3766323e-08
1645 1.29978e-08
1646 1.3766323e-08
1647 1.29978e-08
1648 1.3766323e-08
1649 1.29978e-08
1650 1.3297933e-08
1651 1.0547678e-08
1652 1.1316201e-08
1653 1.0547678e-08
1654 1.1316201e-08
1655 1.0547678e-08
1656 1.1316201e-08
1657 1.0151139e-08
1658 1.0919662e-08
1659 1.0151139e-08
1660 1.0919662e-08
1661 1.0151139e-08
1662 1.0919662e-08
1663 1.0151139e-08
1664 1.

2071 9.169936e-09
2072 8.918461e-09
2073 9.169936e-09
2074 8.918461e-09
2075 9.169936e-09
2076 8.918461e-09
2077 9.169936e-09
2078 8.918461e-09
2079 9.169936e-09
2080 8.918461e-09
2081 9.169936e-09
2082 8.880717e-09
2083 9.132192e-09
2084 8.880717e-09
2085 9.132192e-09
2086 8.880717e-09
2087 9.132192e-09
2088 8.880717e-09
2089 9.132192e-09
2090 8.880717e-09
2091 9.132192e-09
2092 8.880717e-09
2093 9.132192e-09
2094 8.880717e-09
2095 9.132192e-09
2096 8.880717e-09
2097 9.132192e-09
2098 8.880717e-09
2099 9.132192e-09
2100 8.880717e-09
2101 9.132192e-09
2102 8.880717e-09
2103 9.132192e-09
2104 8.880717e-09
2105 9.132192e-09
2106 8.880717e-09
2107 9.132192e-09
2108 8.880717e-09
2109 9.132192e-09
2110 8.880717e-09
2111 9.132192e-09
2112 8.880717e-09
2113 9.132192e-09
2114 8.880717e-09
2115 9.132192e-09
2116 8.880717e-09
2117 9.132192e-09
2118 8.880717e-09
2119 9.132192e-09
2120 8.664143e-09
2121 9.570682e-09
2122 9.319207e-09
2123 9.570682e-09
2124 9.584779e-09
2125 9.570682e-09
2126 9.584

...

8066 2.8026066e-09
8067 2.501109e-09
8068 2.4765527e-09
8069 2.4801907e-09
8070 2.4874667e-09
8071 2.8062446e-09
8072 2.4865572e-09
8073 2.4901952e-09
8074 2.4874667e-09
8075 2.4801907e-09
8076 2.8026066e-09
8077 2.501109e-09
8078 2.4765527e-09
8079 2.4801907e-09
8080 2.4874667e-09
8081 3.0760234e-09
8082 2.3663902e-09
8083 2.4664346e-09
8084 2.4765527e-09
8085 2.4801907e-09
8086 2.8135205e-09
8087 2.4901952e-09
8088 2.4865572e-09
8089 2.4911047e-09
8090 2.4765527e-09
8091 2.8062446e-09
8092 2.4974711e-09
8093 2.4801907e-09
8094 2.4765527e-09
8095 2.4911047e-09
8096 2.8026066e-09
8097 2.4901952e-09
8098 2.4874667e-09
8099 2.4801907e-09
8100 2.4765527e-09
8101 3.0760234e-09
8102 2.3663902e-09
8103 2.476439e-09
8104 2.4874667e-09
8105 2.4801907e-09
8106 2.8026066e-09
8107 2.501109e-09
8108 2.4765527e-09
8109 2.4801907e-09
8110 2.4874667e-09
8111 2.8062446e-09
8112 2.4865572e-09
8113 2.4801907e-09
8114 2.4874667e-09
8115 2.4801907e-09
8116 2.8026066e-09
8117 2.501109e-09
8118 2.4865572e-0

8520 2.4765527e-09
8521 2.4911047e-09
8522 2.8026066e-09
8523 2.9594944e-09
8524 2.4765527e-09
8525 2.4874667e-09
8526 2.4765527e-09
8527 2.8062446e-09
8528 2.4974711e-09
8529 2.4901952e-09
8530 2.4765527e-09
8531 2.4911047e-09
8532 3.0723855e-09
8533 2.8393274e-09
8534 2.4627966e-09
8535 2.4765527e-09
8536 2.4765527e-09
8537 2.8171585e-09
8538 2.4865572e-09
8539 2.4801907e-09
8540 2.4874667e-09
8541 2.4801907e-09
8542 2.8026066e-09
8543 2.9704084e-09
8544 2.3658218e-09
8545 2.7325755e-09
8546 2.6824396e-09
8547 2.476439e-09
8548 2.4874667e-09
8549 2.4801907e-09
8550 2.7463316e-09
8551 2.6860776e-09
8552 2.472801e-09
8553 2.4801907e-09
8554 2.4874667e-09
8555 2.4801907e-09
8556 2.7463316e-09
8557 2.6860776e-09
8558 2.472801e-09
8559 2.501109e-09
8560 2.4765527e-09
8561 2.8062446e-09
8562 2.4865572e-09
8563 2.4911047e-09
8564 2.4765527e-09
8565 2.4801907e-09
8566 2.8135205e-09
8567 2.9594944e-09
8568 2.3558173e-09
8569 2.7325755e-09
8570 2.3563858e-09
8571 2.7362135e-09
8572 2.6824396e-

8970 2.4452889e-09
8971 2.5471523e-09
8972 2.6905111e-09
8973 2.5371478e-09
8974 2.4352844e-09
8975 2.7923746e-09
8976 2.7613383e-09
8977 2.5471523e-09
8978 2.4352844e-09
8979 2.7923746e-09
8980 2.7613383e-09
8981 2.5471523e-09
8982 2.4352844e-09
8983 2.7923746e-09
8984 2.4352844e-09
8985 2.5371478e-09
8986 2.6905111e-09
8987 2.8632017e-09
8988 2.4452889e-09
8989 2.5471523e-09
8990 2.6905111e-09
8991 2.5371478e-09
8992 2.4352844e-09
8993 2.7923746e-09
8994 2.7613383e-09
8995 2.5471523e-09
8996 2.4352844e-09
8997 2.7923746e-09
8998 2.7613383e-09
8999 2.5471523e-09
9000 2.4352844e-09
9001 2.5371478e-09
9002 2.6905111e-09
9003 2.5371478e-09
9004 2.6905111e-09
9005 2.8632017e-09
9006 2.4452889e-09
9007 2.5471523e-09
9008 2.4352844e-09
9009 2.7923746e-09
9010 2.4352844e-09
9011 2.7923746e-09
9012 2.7613383e-09
9013 2.5471523e-09
9014 2.4352844e-09
9015 2.5371478e-09
9016 2.6905111e-09
9017 2.8632017e-09
9018 2.4452889e-09
9019 2.5371478e-09
9020 2.6905111e-09
9021 2.5371478e-09
9022 2.43528

9444 2.0711455e-09
9445 2.1792617e-09
9446 2.0711455e-09
9447 2.1792617e-09
9448 2.1020683e-09
9449 2.1483388e-09
9450 2.1020683e-09
9451 2.1483388e-09
9452 2.1020683e-09
9453 2.1792617e-09
9454 2.0711455e-09
9455 2.1792617e-09
9456 2.0711455e-09
9457 2.1792617e-09
9458 2.0711455e-09
9459 2.1792617e-09
9460 2.1020683e-09
9461 2.1483388e-09
9462 2.1020683e-09
9463 2.1483388e-09
9464 2.1020683e-09
9465 2.1792617e-09
9466 2.0711455e-09
9467 2.1792617e-09
9468 2.0711455e-09
9469 2.1792617e-09
9470 2.0711455e-09
9471 2.1792617e-09
9472 2.1020683e-09
9473 2.1483388e-09
9474 2.1020683e-09
9475 2.1483388e-09
9476 2.1020683e-09
9477 2.1792617e-09
9478 2.0711455e-09
9479 2.1792617e-09
9480 2.0711455e-09
9481 2.1792617e-09
9482 2.0711455e-09
9483 2.1792617e-09
9484 2.1020683e-09
9485 2.1483388e-09
9486 2.1020683e-09
9487 2.1483388e-09
9488 2.1020683e-09
9489 2.1792617e-09
9490 2.0711455e-09
9491 2.1792617e-09
9492 2.0711455e-09
9493 2.1792617e-09
9494 2.0711455e-09
9495 2.1792617e-09
9496 2.10206

9885 1.9620345e-09
9886 1.9693105e-09
9887 1.9824982e-09
9888 1.9693105e-09
9889 1.9620345e-09
9890 1.9896604e-09
9891 1.947369e-09
9892 1.9896604e-09
9893 1.947369e-09
9894 1.9691968e-09
9895 1.9678326e-09
9896 1.9691968e-09
9897 1.947369e-09
9898 1.9896604e-09
9899 1.947369e-09
9900 1.9691968e-09
9901 1.9678326e-09
9902 1.9691968e-09
9903 1.9678326e-09
9904 1.9691968e-09
9905 1.947369e-09
9906 1.9896604e-09
9907 1.947369e-09
9908 1.9691968e-09
9909 1.9678326e-09
9910 1.9691968e-09
9911 1.9678326e-09
9912 1.9691968e-09
9913 1.947369e-09
9914 1.9896604e-09
9915 1.947369e-09
9916 1.9691968e-09
9917 1.9678326e-09
9918 1.9691968e-09
9919 1.947369e-09
9920 1.9896604e-09
9921 1.947369e-09
9922 1.9896604e-09
9923 1.947369e-09
9924 1.970902e-09
9925 1.9678326e-09
9926 1.970902e-09
9927 1.947369e-09
9928 1.9913657e-09
9929 1.947369e-09
9930 1.9913657e-09
9931 1.947369e-09
9932 1.970902e-09
9933 1.9678326e-09
9934 1.970902e-09
9935 1.947369e-09
9936 1.9913657e-09
9937 1.947369e-09
9938 1.970902

In [9]:
now - since

16.758138418197632